In [ ]:
from skimage.filters import threshold_otsu

# approximately solve min_z ||zA-x||^2_2 st ||z||_0 <= max_iters
def matching_pursuit(x, A, max_iters, thr=1):
    # this is necessary if doing more than one iteration, for the residuals update to work
    A = A / np.linalg.norm(A,axis=1)[:,None]

    z = np.zeros((A.shape[0],*x.shape[1:]))

    x = x.copy() # initialize "residuals" to the image itself

    # mask for whether each pixel has converged or not; we already know background's zero
    active_set = np.ones(x.shape[1:],dtype=bool)
    x_norm = np.linalg.norm(x,axis=0,ord=2)

    # one approach: pick it using the otsu threshold, as estimate of noise level
    max_norm = threshold_otsu(x_norm)
#     max_norm = [threshold_otsu(x_norm[k]) for k in range(x_norm.shape[0])]
#     max_norm = np.array(max_norm)[:,None,None]
    max_norm *= thr # hack; otsu just not great. see 'enhance neurites' in CellProfiler?
    active_set[x_norm <= max_norm] = False

    for t in range(max_iters):
        # project dictionary on residual image
        Ax = A @ x[:,active_set]
        # pick index with max projection
        k_max = Ax.argmax(0)
        # set it to active
        z[k_max,active_set] = Ax[k_max,range(len(k_max))]
        # subtract off contribution
        x[:,active_set] -= A[k_max].T * z[k_max,active_set]
        # mark pixels with sufficiently small residual norm as done
        x_norm = np.linalg.norm(x,axis=0,ord=2)
        active_set[x_norm<max_norm] &= False
        
    return z

A = codebook.values.copy().T
x = img.copy()
assert len(x.shape) == 4

max_components = 3
fudge_factor = 0.25

z = matching_pursuit(x, A, max_components, fudge_factor)

z = z.clip(0,np.inf)

z.shape